<a href="https://colab.research.google.com/github/AlirezaParchami/Real-time-Character-Detection-using-Faster-RCNN/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies and Detectron2

In [ ]:
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [ ]:
#Download Character Dataset and unzip
#!wget https://github.com/MinhasKamal/AlphabetRecognizer/files/1084725/English.Alphabet.Dataset.zip
!unzip English.Alphabet.Dataset.zip

In [3]:
#Version 1
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog

def get_character_dict(img_dir, num):
  dir = img_dir + str(num)+ "/" + str(num) + ".json"
  json_file = os.path.join(dir)
  
  with open(json_file) as f:
    imgs_anns = json.load(f)
  
  character_dicts = []
  for idx, v in zip(imgs_anns.keys(), imgs_anns.values()):
    record = {}

    filename = os.path.join(img_dir + str(num) + "/" + v["filename"])
    height, width = cv2.imread(filename).shape[:2]

    record["file_name"] = filename
    record["image_id"] = idx
    record["height"] = height
    record["width"] = width

    annos = v["regions"]
    objs = []
    for _, anno in annos.items():
      anno = anno["shape_attributes"]
      px = anno["points_x"]
      py = anno["points_y"]
      poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      poly = [p for x in poly for p in x]

      obj = {
          "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
          "bbox_mode": BoxMode.XYXY_ABS,
          "segmentation": [poly],
          "category_id": num - 1,
      }
      objs.append(obj)
    record["annotations"] = objs
    character_dicts.append(record)
  return character_dicts
  

for d in range(1,8):
  DatasetCatalog.register("character_train_" + str(d), lambda d=d: get_character_dict("English Alphabet Dataset/", d))
  MetadataCatalog.get("character_train_" + str(d)).set(thing_classes=[str(d)])
character_metadata = MetadataCatalog.get("character_train")
print("Finished Successfully")

Finished Successfully


In [3]:
#Version 2
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog

def get_character_dict(img_dir):
  dataset_dicts = []
  for num in range(1,8):
    dir = img_dir + str(num)+ "/" + str(num) + ".json"
    json_file = os.path.join(dir)
  
    with open(json_file) as f:
      imgs_anns = json.load(f)
  
    character_dicts = []
    for idx, v in zip(imgs_anns.keys(), imgs_anns.values()):
      record = {}

      filename = os.path.join(img_dir + str(num) + "/" + v["filename"])
      height, width = cv2.imread(filename).shape[:2]

      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      annos = v["regions"]
      objs = []
      for _, anno in annos.items():
        anno = anno["shape_attributes"]
        px = anno["points_x"]
        py = anno["points_y"]
        poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
        poly = [p for x in poly for p in x]

        obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly],
            "category_id": num - 1,
        }
        objs.append(obj)
      record["annotations"] = objs
      character_dicts.append(record)

    dataset_dicts.extend(character_dicts)
  return dataset_dicts
  

for d in ["train"]:
  DatasetCatalog.register("character_" + d, lambda d=d: get_character_dict("English Alphabet Dataset/" + d + "/"))
  MetadataCatalog.get("character_" + str(d)).set(thing_classes=["1","2","3","4","5","6","7"])
character_metadata = MetadataCatalog.get("character_train")
print("Finished Successfully")

Finished Successfully


In [7]:
#Check if the metadata is added successfully

from detectron2.utils.visualizer import Visualizer
dataset_dicts = get_character_dict("English Alphabet Dataset/train/")

for d in random.sample(dataset_dicts, 3):
  print(d)
  img = cv2.imread(d["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=character_metadata, scale=0.5)
  out = visualizer.draw_dataset_dict(d)
  cv2_imshow(out.get_image()[:, :, ::-1])

here
30
60
90
120
150
180
210
210
5
{'file_name': 'English Alphabet Dataset/train/1/1_2.png', 'image_id': '1_2', 'height': 60, 'width': 60, 'annotations': [{'bbox': [2, 6, 58, 56], 'bbox_mode': <BoxMode.XYXY_ABS: 0>, 'segmentation': [[2.5, 6.5, 58.5, 56.5]], 'category_id': 0}]}
{'file_name': 'English Alphabet Dataset/train/3/3_29.png', 'image_id': '3_29', 'height': 60, 'width': 60, 'annotations': [{'bbox': [2, 6, 58, 56], 'bbox_mode': <BoxMode.XYXY_ABS: 0>, 'segmentation': [[2.5, 6.5, 58.5, 56.5]], 'category_id': 2}]}


{'file_name': 'English Alphabet Dataset/train/2/2_30.png', 'image_id': '2_30', 'height': 60, 'width': 60, 'annotations': [{'bbox': [1, 3, 58, 57], 'bbox_mode': <BoxMode.XYXY_ABS: 0>, 'segmentation': [[1.5, 3.5, 58.5, 57.5]], 'category_id': 1}]}


{'file_name': 'English Alphabet Dataset/train/5/5_18.png', 'image_id': '5_18', 'height': 60, 'width': 60, 'annotations': [{'bbox': [1, 6, 58, 56], 'bbox_mode': <BoxMode.XYXY_ABS: 0>, 'segmentation': [[1.5, 6.5, 58.5, 56.5]], 'category_id': 4}]}


**Train the Model**

Want to fine-tune a COCO-pretrained R50-FPN 3x with model reference number of 137849458 for Faster RCNN






In [11]:
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
#cfg.DATASETS.TRAIN = ("character_train_1","character_train_2", "character_train_3", "character_train_4", "character_train_5","character_train_6","character_train_7",)
cfg.DATASETS.TRAIN = ("character_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0002
cfg.SOLVER.MAX_ITER = 40
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7


In [12]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[01/19 07:25:00 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:06, 24.5MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (28, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (28,) in the model! You might want to double check if this is expected.


[01/19 07:25:12 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.6/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:217: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[01/19 07:25:19 d2.utils.events]:  eta: 0:00:06  iter: 19  total_loss: 2.111  loss_cls: 1.994  loss_box_reg: 0.09491  loss_rpn_cls: 0.004397  loss_rpn_loc: 0.006706  time: 0.3354  data_time: 0.0128  lr: 3.9962e-06  max_mem: 1991M
[01/19 07:25:27 d2.utils.events]:  eta: 0:00:00  iter: 39  total_loss: 1.871  loss_cls: 1.757  loss_box_reg: 0.09663  loss_rpn_cls: 0.003651  loss_rpn_loc: 0.006932  time: 0.3392  data_time: 0.0055  lr: 7.9922e-06  max_mem: 1991M
[01/19 07:25:28 d2.engine.hooks]: Overall training speed: 38 iterations in 0:00:12 (0.3393 s / it)
[01/19 07:25:28 d2.engine.hooks]: Total training time: 0:00:14 (0:00:01 on hooks)


**Evaluate**

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)